In [4]:
import requests
import pandas as pd
import json
import os
import shutil
from openpyxl import load_workbook

# ======================
# CARGA DE CREDENCIALES
# ======================
def load_blizzard_credentials(path):
    creds = {}
    print(f"🔐 Cargando credenciales desde: {path}")
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            key, value = line.split("=", 1)
            creds[key.strip()] = value.strip()
    if "CLIENT_ID" not in creds or "CLIENT_SECRET" not in creds:
        raise RuntimeError("❌ Credenciales inválidas")
    print("✅ Credenciales cargadas")
    return creds["CLIENT_ID"], creds["CLIENT_SECRET"]

# ======================
# DETECCIÓN DE ESCRITORIO
# ======================
def find_desktop():
    home = os.path.expanduser("~")
    candidates = [
        os.path.join(home, "OneDrive", "Desktop"),
        os.path.join(home, "OneDrive", "Escritorio"),
    ]
    for path in candidates:
        if os.path.exists(path):
            print(f"📁 Escritorio detectado: {path}")
            return path
    raise RuntimeError("❌ No se encontró el Escritorio")

DESKTOP_PATH = find_desktop()
CREDENTIALS_FILE = os.path.join(DESKTOP_PATH, "blizzard_credentials.txt")
CLIENT_ID, CLIENT_SECRET = load_blizzard_credentials(CREDENTIALS_FILE)

# ======================
# CONFIG
# ======================
REGION = "us"
LOCALE = "en_US"

PRIMARY_REALMS = ["Stormrage", "Area 52", "Moon Guard", "Ragnaros","Dalaran","Zul'jin","Proudmoore"]

INPUT_XLSX = "Book.xlsx"
OUTPUT_XLSX = "BookArbitrage.xlsx"
SHEET_NAME = "Hoja3"
CACHE_FILE = "item_id_cache.json"
REALMS_JSON = os.path.join(DESKTOP_PATH, "wow_connected_realms.json")

# ======================
# AUTENTICACIÓN
# ======================
def get_token():
    print("🔐 Obteniendo token Blizzard...")
    r = requests.post(
        f"https://{REGION}.battle.net/oauth/token",
        auth=(CLIENT_ID, CLIENT_SECRET),
        data={"grant_type": "client_credentials"}
    )
    r.raise_for_status()
    print("✅ Token obtenido")
    return r.json()["access_token"]

# ======================
# CACHE
# ======================
def load_cache():
    return json.load(open(CACHE_FILE)) if os.path.exists(CACHE_FILE) else {}

def save_cache(cache):
    json.dump(cache, open(CACHE_FILE, "w"), indent=2)
    print("💾 Cache guardada")

# ======================
# COPIA DE EXCEL
# ======================
def create_excel_copy(src, dst):
    shutil.copy(src, dst)
    print(f"📄 Copia creada: {dst}")

# ======================
# UBICACIÓN DE TABLA
# ======================
def find_enchanting_table(path, sheet_name):
    raw = pd.read_excel(path, sheet_name=sheet_name, header=None)
    for r in range(len(raw)):
        row = raw.iloc[r].astype(str).str.strip().str.lower().tolist()
        if "item" in row and "quantity" in row and "estimated sell price" in row:
            return r + 1, row.index("item") + 1, row.index("quantity") + 1, row.index("estimated sell price") + 1
    raise RuntimeError("❌ Tabla Enchanting no encontrada")

# ======================
# FILTRAR REALMS
# ======================
def load_realms(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    realms = {}
    for r in data:
        name = r.get("name", "")
        if any(skip in name.upper() for skip in ["AU", "AUXILIARY", "INST", "TEST"]):
            continue
        realms[name] = r["id"]
    return realms

# ======================
# BÚSQUEDA DE ITEM ID
# ======================
def get_item_id(token, item_name, cache):
    if item_name in cache:
        return cache[item_name]
    r = requests.get(
        f"https://{REGION}.api.blizzard.com/data/wow/search/item",
        headers={"Authorization": f"Bearer {token}"},
        params={"namespace": f"static-{REGION}", "locale": LOCALE, "name.en_US": item_name, "_pageSize": 10}
    )
    r.raise_for_status()
    for res in r.json().get("results", []):
        name = res["data"]["name"].get(LOCALE)
        if name and name.lower() == item_name.lower():
            cache[item_name] = res["data"]["id"]
            return cache[item_name]
    return None

# ======================
# SUBASTAS
# ======================
def get_all_auctions(token, realms):
    all_auctions = {}
    for realm, realm_id in realms.items():
        print(f"📡 Descargando subastas de {realm}...")
        r = requests.get(
            f"https://{REGION}.api.blizzard.com/data/wow/connected-realm/{realm_id}/auctions",
            headers={"Authorization": f"Bearer {token}"},
            params={"namespace": f"dynamic-{REGION}", "locale": LOCALE}
        )
        if r.status_code != 200:
            print(f"   ⚠ Error al obtener {realm}: {r.status_code}")
            continue
        all_auctions[realm] = r.json().get("auctions", [])
    return all_auctions

def get_price_per_unit(a):
    if a.get("unit_price"):
        return a["unit_price"]
    if a.get("buyout"):
        return a["buyout"] / a.get("quantity", 1)
    return None

def min_buyout(auctions, item_id):
    prices = [get_price_per_unit(a) for a in auctions if a.get("item", {}).get("id") == item_id and get_price_per_unit(a)]
    return min(prices) if prices else None

def min_buyout_by_realm(all_auctions, item_id):
    prices = {}
    for realm, auctions in all_auctions.items():
        p = min_buyout(auctions, item_id)
        if p:
            prices[realm] = p
    return prices

# ======================
# LÓGICA DE ARBITRAJE
# ======================
def analyze_arbitrage(prices):
    if not prices:
        return None, None, None
    buy_realm = min(prices, key=prices.get)
    buy_price = prices[buy_realm]
    best_target = None
    best_profit = 0
    for realm in PRIMARY_REALMS:
        if realm in prices:
            sell_price = prices[realm] * 0.95
            profit = sell_price - buy_price
            if profit > best_profit:
                best_profit = profit
                best_target = realm
    if best_profit < 1000:
        return buy_realm, None, 0
    return buy_realm, best_target, best_profit

# ======================
# MAIN
# ======================
def main():
    create_excel_copy(INPUT_XLSX, OUTPUT_XLSX)
    token = get_token()
    realms = load_realms(REALMS_JSON)
    print(f"🌐 Reinos válidos cargados: {list(realms.keys())[:10]} ...")
    all_auctions = get_all_auctions(token, realms)
    cache = load_cache()

    header_row, col_item, col_qty, col_price = find_enchanting_table(OUTPUT_XLSX, SHEET_NAME)
    wb = load_workbook(OUTPUT_XLSX)
    ws = wb[SHEET_NAME]

    col_best_buy = col_price + 1
    col_target_realm = col_price + 2
    col_profit = col_price + 3
    ws.cell(row=header_row, column=col_best_buy, value="Best Buy Realm")
    ws.cell(row=header_row, column=col_target_realm, value="Target Realm")
    ws.cell(row=header_row, column=col_profit, value="Profit")

    r = header_row + 1
    updated = 0

    print("\n🚀 Analizando arbitraje multi-reino...\n")
    while True:
        item_cell = ws.cell(row=r, column=col_item)
        qty_cell = ws.cell(row=r, column=col_qty)
        price_cell = ws.cell(row=r, column=col_price)

        if not item_cell.value:
            break

        quantity = int(qty_cell.value or 0)
        if quantity == 0:
            print(f"⏭ {item_cell.value} (Quantity = 0)")
            r += 1
            continue

        item_name = str(item_cell.value).strip()
        item_id = get_item_id(token, item_name, cache)
        if not item_id:
            print(f"   ❌ Item no encontrado: {item_name}")
            r += 1
            continue

        prices = min_buyout_by_realm(all_auctions, item_id)
        buy_realm, target_realm, profit = analyze_arbitrage(prices)

        if target_realm:
            price_cell.value = round(prices[target_realm] / 10000, 2)
            ws.cell(row=r, column=col_best_buy, value=buy_realm)
            ws.cell(row=r, column=col_target_realm, value=target_realm)
            ws.cell(row=r, column=col_profit, value=round(profit / 10000, 2))
            print(f"➡ {item_name}")
            print(f"   📊 Precios:")
            for realm, p in prices.items():
                print(f"      {realm}: {round(p/10000,2)}g")
            print(f"   🏆 Comprar en {buy_realm}, vender en {target_realm} → Profit {round(profit/10000,2)}g")
            updated += 1
        else:
            print(f"➡ {item_name} → No rentable en PRIMARY_REALMS")

        r += 1

    save_cache(cache)
    wb.save(OUTPUT_XLSX)

    print("\n============================")
    print(f"✔ {updated} items actualizados")
    print(f"✔ Archivo generado: {OUTPUT_XLSX}")
    print("============================")

main()


📁 Escritorio detectado: C:\Users\diego\OneDrive\Desktop
🔐 Cargando credenciales desde: C:\Users\diego\OneDrive\Desktop\blizzard_credentials.txt
✅ Credenciales cargadas
📄 Copia creada: BookArbitrage.xlsx
🔐 Obteniendo token Blizzard...
✅ Token obtenido
🌐 Reinos válidos cargados: ['Azjol-Nerub', 'Firetree', 'Runetotem', 'Draenor', 'Aggramar', 'Ravencrest', 'Stormreaver', 'Elune', 'Bleeding Hollow', 'Trollbane'] ...
📡 Descargando subastas de Azjol-Nerub...
📡 Descargando subastas de Firetree...
📡 Descargando subastas de Runetotem...
📡 Descargando subastas de Draenor...
📡 Descargando subastas de Aggramar...
📡 Descargando subastas de Ravencrest...
📡 Descargando subastas de Stormreaver...
📡 Descargando subastas de Elune...
📡 Descargando subastas de Bleeding Hollow...
📡 Descargando subastas de Trollbane...
📡 Descargando subastas de Vek'nilash...
📡 Descargando subastas de Sen'jin...
📡 Descargando subastas de Argent Dawn...
📡 Descargando subastas de Dragonmaw...
📡 Descargando subastas de Silvermo

In [2]:
realms = load_realms(REALMS_JSON)
print(realms["Dalaran"])


66
